# Script to make contour plot in python and overlay with raw signal
Rhodes Ford
Started Feb 6, 2024

In [1]:
# import needed libraries

import os

import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from alpineer import io_utils

from PIL import Image

import skimage as ski

In [2]:
# Specify files/information for input
## 
tiff_dir = '/c4/home/brford/HNSCC/HNLN/raw_data/'
markers = ['PD-1','PD-L1']
output_dir = '/c4/home/brford/HNSCC/HNLN/rlVectorAnalysis/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.path.exists(output_dir + 'contour'):
    os.makedirs(output_dir + 'contour')
    
if not os.path.exists(output_dir + 'png'):
    os.makedirs(output_dir + 'png')
    
if not os.path.exists(output_dir + 'overlay'):
    os.makedirs(output_dir + 'overlay')

In [3]:
# Dictionary to store images with the same name
image_dict = {}


# Get a list of all directories (one per FOV) in parent directory
fovs = os.listdir(tiff_dir)

# Iterate through each marker and TIFF directory and make dictionary of PD-1 and PD-L1 signal for each X/Y coordinate for each FOV

for fov in fovs:
    imdf = pd.DataFrame()
    for i in markers:
        marker_tiff_name = i + '.tiff'
        
        
        file_name = os.path.join(tiff_dir + fov + '/' + marker_tiff_name)
        print(file_name)
        image = Image.open(file_name)
        imarray = np.array(image)
        
        result = []
        
        h, w = imarray.shape
        for x in range(w):
            for y in range(h):
                result.append([x, y, imarray[x,y]])
        
        imdf_temp = pd.DataFrame(result,columns = ['X','Y',i])
        if 'X' in imdf:
            imdf = imdf.merge(imdf_temp, on = ['X','Y'])
        else:
            imdf = imdf_temp
    
    image_dict[fov] = imdf
    

/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-09T08-59-38-fov-1-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-09T08-59-38-fov-1-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-17T13-45-35-fov-2-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-17T13-45-35-fov-2-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-21T22-45-42-fov-4-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-21T22-45-42-fov-4-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-18T16-29-36-fov-5-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-18T16-29-36-fov-5-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-21T11-08-40-fov-1-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-21T11-08-40-fov-1-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-06T22-21-28-fov-2-scan-1/PD-1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-06T22-21-28-fov-2-scan-1/PD-L1.tiff
/c4/home/brford/HNSCC/HNLN/raw_data/2022-03-20

In [22]:
for fov in image_dict:
    for i in markers:
        print(i + " in " + fov)
        
        fov_df = image_dict[fov]
        # Pivot the DataFrame
        pivot_df = fov_df.pivot(index='X', columns='Y', values=i)

        # Convert the pivot DataFrame to a NumPy array
        numpy_array = pivot_df.to_numpy()
        
        file_name = "".join([fov,'_',i,'_contour.tiff'])
        print(file_name)
        # make contour plot
        plt.figure(figsize=[20.48,20.48])
        contour = plt.contour(numpy_array,[0.00275,0.00375,0.00475],colors = 'cyan',linewidths=0.1,origin = 'upper')

        plt.subplot()
        plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
        plt.axis('off')        

        plt.savefig(os.path.join(output_dir,'contour',file_name),format='tiff',transparent=True,dpi=500)
        plt.close()
        
        image_1 = Image.open(os.path.join(output_dir,'contour',file_name))
        resized_img = image_1.resize((2048, 2048), Image.ANTIALIAS)
        
        # Open the original TIFF to overlay this contour plot with
        image_2 = Image.open(os.path.join(tiff_dir,fov,i+'.tiff'))
        imarray = np.array(image_2)
        amplified_array = np.multiply(imarray, 10000)
        amplified_image = Image.fromarray(amplified_array)
        amplified_image = amplified_image.convert("RGBA")
        amplified_image.save(os.path.join(output_dir,'png',fov+'_'+i+'.png'), format='PNG')
        
        # Merge the images
        blend = Image.blend(amplified_image,resized_img,0.5)
        plt.imshow(blend)
        
        plt.subplot()
        plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
        plt.axis('off')        

        plt.savefig(os.path.join(output_dir,'overlay',fov + "_" + i + ".png"),format='png',transparent=True,dpi=500)
        plt.close()

        print('cycle completed for: ' + i + " in " + fov)

PD-1 in 2022-03-09T08-59-38-fov-1-scan-1
2022-03-09T08-59-38-fov-1-scan-1_PD-1_contour.tiff


/scratch/brford/1680556/ipykernel_118094/1071517061.py:26: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = image_1.resize((2048, 2048), Image.ANTIALIAS)


cycle completed for: PD-1 in 2022-03-09T08-59-38-fov-1-scan-1
PD-L1 in 2022-03-09T08-59-38-fov-1-scan-1
2022-03-09T08-59-38-fov-1-scan-1_PD-L1_contour.tiff
cycle completed for: PD-L1 in 2022-03-09T08-59-38-fov-1-scan-1
PD-1 in 2022-03-17T13-45-35-fov-2-scan-1
2022-03-17T13-45-35-fov-2-scan-1_PD-1_contour.tiff
cycle completed for: PD-1 in 2022-03-17T13-45-35-fov-2-scan-1
PD-L1 in 2022-03-17T13-45-35-fov-2-scan-1
2022-03-17T13-45-35-fov-2-scan-1_PD-L1_contour.tiff
cycle completed for: PD-L1 in 2022-03-17T13-45-35-fov-2-scan-1
PD-1 in 2022-03-21T22-45-42-fov-4-scan-1
2022-03-21T22-45-42-fov-4-scan-1_PD-1_contour.tiff
cycle completed for: PD-1 in 2022-03-21T22-45-42-fov-4-scan-1
PD-L1 in 2022-03-21T22-45-42-fov-4-scan-1
2022-03-21T22-45-42-fov-4-scan-1_PD-L1_contour.tiff
cycle completed for: PD-L1 in 2022-03-21T22-45-42-fov-4-scan-1
PD-1 in 2022-03-18T16-29-36-fov-5-scan-1
2022-03-18T16-29-36-fov-5-scan-1_PD-1_contour.tiff
cycle completed for: PD-1 in 2022-03-18T16-29-36-fov-5-scan-1
PD-L1